**Author:**       Jensy Gregorio Gómez 
---------------------------------------
**Profession:**  IT Support Analyst | NOC | Database Administrator

**Date:**         20 Junio 2024    

**Modified**  24 Junio 2024

**Location:**     Vila Izabel, Curitiba/PR  


---

**Contacto:**

- **Email:** [contact@jensygomez.us](mailto:contact@jensygomez.us)
- **YouTube:** [Tu Canal de YouTube](https://www.youtube.com/@systechcwb826)
- **LinkedIn:** [Tu Perfil de LinkedIn](https://www.linkedin.com/in/jensygomez/)







### Título del Ejercicio:

**"Identificación de Empleados con Órdenes Excepcionales"**

### Orientación del Ejercicio:

Este ejercicio está diseñado para evaluar tu habilidad para trabajar con subconsultas avanzadas en SQL. Te enfrentarás a un escenario donde necesitas identificar a los empleados que han gestionado órdenes cuyo monto total supera el promedio de los montos totales de órdenes gestionadas por otros empleados en la misma oficina. Este tipo de análisis es crucial para entender la eficiencia y el desempeño de los empleados en diferentes oficinas.

#### Pasos a Seguir:

1.  **Comprender las Tablas Involucradas:**
    
    -   `employees`: Contiene información sobre los empleados, incluyendo el código de oficina (`officeCode`).
    -   `offices`: Contiene información sobre las oficinas, incluyendo la ciudad (`city`).
    -   `orders` y `orderdetails`: Contienen información sobre las órdenes y sus detalles, respectivamente.
    -   `customers`: Relaciona los clientes con sus representantes de ventas (empleados).
2.  **Construir Subconsultas:**
    
    -   Necesitas una subconsulta para calcular el monto total de cada orden. Esto se hace sumando el producto de `quantityOrdered` y `priceEach` de la tabla `orderdetails` agrupada por `orderNumber`.
    -   Otra subconsulta debe calcular el promedio de los montos totales de órdenes gestionadas por otros empleados en la misma oficina. Esto se logra mediante una subconsulta anidada que utiliza la misma lógica de cálculo de montos totales de órdenes.
3.  **Filtrar Resultados:**
    
    -   Filtra los empleados cuyas órdenes tienen un monto total mayor que el promedio calculado en la subconsulta anterior.
4.  **Unir Tablas:**
    
    -   Usa las claves foráneas (`officeCode`, `salesRepEmployeeNumber`) para unir las tablas `employees`, `offices`, `customers`, `orders` y `orderdetails`.
5.  **Seleccionar Campos Relevantes:**
    
    -   Asegúrate de seleccionar los nombres de los empleados, la ciudad de la oficina y el monto total de las órdenes que cumplen con la condición.

### Ejercicio:

Escribe una consulta SQL para obtener los nombres de los empleados que han gestionado órdenes cuyo monto total es mayor que el promedio de los montos totales de órdenes gestionadas por otros empleados en la misma oficina. La consulta también debe mostrar el nombre de la ciudad de la oficina y el monto total de esas órdenes.

In [ ]:
-- Selecciona el nombre y apellido de los empleados
SELECT 
    employees.firstName,
    employees.lastName,
    -- Selecciona el nombre de la ciudad de la oficina
    offices.city AS officeCity,
    -- Selecciona el monto total de las órdenes gestionadas por el empleado
    orderTotals.totalOrderAmount
FROM 
    -- Une la tabla de empleados
    employees
JOIN 
    -- Une la tabla de oficinas basada en el código de oficina
    offices ON employees.officeCode = offices.officeCode
JOIN 
    (
        -- Subconsulta para calcular el monto total de cada orden
        SELECT 
            orders.customerNumber,
            orders.orderNumber,
            SUM(orderdetails.quantityOrdered * orderdetails.priceEach) AS totalOrderAmount
        FROM 
            -- Une la tabla de órdenes
            orders
        JOIN 
            -- Une la tabla de detalles de órdenes
            orderdetails ON orders.orderNumber = orderdetails.orderNumber
        GROUP BY 
            -- Agrupa por número de orden
            orders.orderNumber
    ) AS orderTotals ON employees.employeeNumber = 
        -- Subconsulta para obtener el número de empleado del representante de ventas del cliente
        (SELECT customers.salesRepEmployeeNumber FROM customers WHERE customers.customerNumber = orderTotals.customerNumber)
WHERE 
    -- Condición para filtrar órdenes con monto total mayor al promedio
    orderTotals.totalOrderAmount > (
        SELECT 
            -- Calcula el promedio de los montos totales de órdenes en la misma oficina
            AVG(orderAverages.totalOrderAmount)
        FROM 
            (
                -- Subconsulta para calcular el monto total de cada orden
                SELECT 
                    orders.customerNumber,
                    orders.orderNumber,
                    SUM(orderdetails.quantityOrdered * orderdetails.priceEach) AS totalOrderAmount
                FROM 
                    -- Une la tabla de órdenes
                    orders
                JOIN 
                    -- Une la tabla de detalles de órdenes
                    orderdetails ON orders.orderNumber = orderdetails.orderNumber
                GROUP BY 
                    -- Agrupa por número de orden
                    orders.orderNumber
            ) AS orderAverages
        JOIN 
            -- Une la tabla de clientes
            customers ON orderAverages.customerNumber = customers.customerNumber
        JOIN 
            -- Une la tabla de empleados con alias e2
            employees AS e2 ON customers.salesRepEmployeeNumber = e2.employeeNumber
        WHERE 
            -- Filtra por oficina del empleado
            e2.officeCode = employees.officeCode
    );

